<a href="https://colab.research.google.com/github/rdion-ken/mbti-text-analysis/blob/main/notebook/MBTI_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Psychology of Social Media Behavior and Online Personas

## 1. Overview and Goal

This project explores how **personality traits shape language patterns in online self expression**.

Using the **MBTI 500 Personality Types Dataset**, the study analyzes how distinct personality categories correspond to unique linguistic signatures.

Rather than focusing on tone or syntax, the project emphasizes **quantitative text analysis and machine learning classification.**

By examining the relationship between MBTI dimensions and text features, it uncovers whether personality differences can be inferred from the words people use, even after text standardization.

Specifically, this study also aims to:
- Identify **linguistic features** that distinguish personality types.
- Train and evaluate a **text classification model** capable of predicting MBTI traits from language data.
- Visualize the relationships between **personality dimensions** and **dominant lexical patterns** within the dataset.

## 2. Dataset Description

**Dataset:** MBTI 500 Personality Types Dataset  
**Source:** [Kaggle – MBTI 500 Dataset](https://www.kaggle.com/datasets/zeyadkhalid/mbti-personality-types-500-dataset/data) (Zeyad Khalid, 2021)  

This dataset contains approximately 106,000 preprocessed records of user posts paired with MBTI personality types, stored in `mbti_500.csv`

**Structure:**
- `type` → 16 MBTI personality type (e.g. INTJ, ENFP)  
- `posts` → 500 word text sample from Reddit/PersonalityCafe  

**Timeframe:** Posts originally gathered between 2016–2019.  Cleaned in 2021.  

Since the text has been preprocessed punctuation, stopwords, and URLs removed, and all samples standardized to equal length, this dataset is particularly well suited for **machine learning classification tasks.**

It provides a large dataset ideal for identifying **lexical patterns** and **predictive linguistic features** associated with personality types.

## 3. Methodology and Approach

### Step 1: Data Access and Library Import
Import core libraries and download the MBTI dataset directly for reproducibility.


In [ ]:
# Import core libraries
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning tools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# For linguistic insights later
import nltk
from nltk import pos_tag
from collections import Counter
nltk.download('averaged_perceptron_tagger')

# Display options
pd.set_option('display.max_colwidth', 150)
pd.options.display.float_format = '{:.2f}'.format

# Download MBTI dataset
url = "https://drive.google.com/uc?id=1KM-1SAIhKhLhdQawPKBoyR3vGBCTJPzV"
output = "mbti_500.csv"

print("Downloading dataset...")
gdown.download(url, output, quiet=False)

# Load dataset
try:
    data = pd.read_csv(output)
    print(f"Dataset loaded successfully. Shape: {data.shape}")
    print("Columns:", list(data.columns))
except Exception as e:
    print("Error loading dataset:", e)

### Step 2: Data Cleaning and Feature Engineering

Since the text in this dataset has already been lemmatized, standardized, and stripped of punctuation, stopwords, and URLs, it is well suited for machine learning based text classification.

This stage focuses on preparing the dataset for modeling by ensuring data integrity and engineering structured features from the MBTI types.


1. Remove duplicates and missing values to ensure dataset consistency.
2. Verify text normalization, confirming that all entries are lowercase and alphabetic.
3. Split MBTI personality types into four binary columns representing the individual psychological dimensions:
*   Introversion (I) vs. Extraversion (E)
*   Sensing (S) vs. Intuition (N)
*   Thinking (T) vs. Feeling (F)
*   Judging (J) vs. Perceiving (P)
4. Prepare the processed dataset for vectorization and machine learning classification in the next step.

In [ ]:
import re
from tqdm.notebook import tqdm
tqdm.pandas(desc="Processing text features")

# Remove duplicates and missing values
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

# Split MBTI types into binary dimensions
data["I_E"] = data["type"].apply(lambda x: 1 if "I" in x else 0)  # 1 = Introvert, 0 = Extrovert
data["S_N"] = data["type"].apply(lambda x: 1 if "S" in x else 0)  # 1 = Sensing, 0 = Intuitive
data["T_F"] = data["type"].apply(lambda x: 1 if "T" in x else 0)  # 1 = Thinking, 0 = Feeling
data["J_P"] = data["type"].apply(lambda x: 1 if "J" in x else 0)  # 1 = Judging, 0 = Perceiving

# Overview of class balance
print("Data cleaning completed.")
print(f"Dataset shape after cleaning: {data.shape}")
print("\nPersonality type distribution:")
print(data['type'].value_counts().head(10))

# Preview a few samples
display(data[["type", "posts"]].head())


### Step 3: Save Cleaned Dataset
After verifying data integrity and preparing MBTI dimension columns, we export the cleaned dataset for reuse and modeling.


In [ ]:
data.to_csv("clean_mbti_500.csv", index=False)
print("Cleaned dataset saved as clean_mbti_500.csv")


### Step 4: Analyze N-gram Frequencies

This step explores the most common word patterns (unigrams, bigrams, trigrams) used by different personality types.

By identifying frequent word combinations, we can infer stylistic and thematic tendencies in how individuals of varying MBTI types express themselves.

Even though the dataset is preprocessed and standardized, word frequency patterns can still reveal meaningful **lexical preferences** (“feel like” for Feeling types vs. “think about” for Thinking types).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
import pandas as pd

# Load the cleaned dataset
data = pd.read_csv("clean_mbti_500.csv")

def get_top_ngram(corpus, n=None, num_words=10):
    vec = CountVectorizer(stop_words='english', ngram_range=(n, n))
    bag = vec.fit_transform(corpus)
    sum_words = bag.sum(axis=0)
    words_freq = [(word, int(sum_words[0, idx])) for word, idx in vec.vocabulary_.items()]
    return sorted(words_freq, key=lambda x: x[1], reverse=True)[:num_words]

ngram_analysis = {}

# Use tqdm progress bar for each MBTI type
for mbti_type in tqdm(data['type'].unique(), desc="Analyzing MBTI Types"):
    type_posts = data.loc[data['type'] == mbti_type, 'posts']
    top_bigrams = get_top_ngram(type_posts, n=2, num_words=10)
    top_trigrams = get_top_ngram(type_posts, n=3, num_words=10)
    ngram_analysis[mbti_type] = {
        'bigrams': top_bigrams,
        'trigrams': top_trigrams
    }

# Display sample output
for mbti_type in list(ngram_analysis.keys())[:3]:
    print(f"\nTop N-grams for {mbti_type}:")
    print("  Bigrams:", ngram_analysis[mbti_type]['bigrams'])
    print("  Trigrams:", ngram_analysis[mbti_type]['trigrams'])
    print("-" * 40)


### Step 5: Part of Speech Frequency Analysis

The goal here is to quantify how each MBTI type uses different grammatical categories nouns, verbs, adjectives, pronouns, etc. using a representative sample of posts (~500 per MBTI type).

*   Intuitives might use more adjectives/adverbs (abstract or descriptive language).
*   Thinkers might favor nouns and verbs (analytical, direct phrasing).
*   Feelers might retain more pronouns (relational focus).


Approch:
*   Apply spaCy to compute POS ratios per post.
*   Aggregate ratios by MBTI type for comparative insights.




In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import spacy
from collections import Counter

# Load cleaned dataset
data = pd.read_csv("clean_mbti_500.csv")

# Stratified sampling: ~500 posts per MBTI type
sampled_data = data.groupby('type', group_keys=False).apply(lambda x: x.sample(min(len(x), 500), random_state=42)).reset_index(drop=True)
print(f"Sampled dataset shape: {sampled_data.shape}")

# Load spaCy model
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

# Define POS categories
pos_map = {
    "NOUN": ["NOUN", "PROPN"],
    "VERB": ["VERB", "AUX"],
    "ADJ": ["ADJ"],
    "ADV": ["ADV"],
    "PRON": ["PRON"]
}

# Function to compute POS ratios
def get_pos_ratios(text):
    doc = nlp(text)
    counts = Counter()
    for token in doc:
        for pos_category, spacy_tags in pos_map.items():
            if token.pos_ in spacy_tags:
                counts[pos_category] += 1
                break
    total = sum(counts.values()) or 1
    return {f"{pos}_ratio": counts[pos]/total for pos in pos_map.keys()}

# Apply POS analysis to the sampled dataset
tqdm.pandas(desc="POS analysis")
pos_features = sampled_data['posts'].progress_apply(get_pos_ratios)
pos_df = pd.DataFrame(list(pos_features))

# Combine results with MBTI types
sampled_pos_data = pd.concat([sampled_data.reset_index(drop=True), pos_df], axis=1)

# Aggregate by MBTI type
pos_summary = sampled_pos_data.groupby("type")[["NOUN_ratio","VERB_ratio","ADJ_ratio","ADV_ratio","PRON_ratio"]].mean()
print("POS Frequency Summary (by MBTI type):")
display(pos_summary)


### Step 6: TF-IDF Vectorization + Simple Classifier

Transforms the preprocessed text into a numerical representation using TF-IDF and train a simple classifier to predict MBTI personality traits. This serves as a proof of concept to show whether language patterns are predictive of personality types.



*   Use TF-IDF to vectorize text while capturing term importance.
*   Train a Logistic Regression model for multi-class classification.
*   Evaluate using accuracy and confusion matrix for interpretability.
*   Can optionally run on a subset for faster experimentation in Colab.






In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm.auto import tqdm

# Load cleaned dataset
data = pd.read_csv("clean_mbti_500.csv")

# Optional: use a subset for faster testing (like 5000 posts)
# data = data.sample(5000, random_state=42)

# TF-IDF vectorization
tqdm.pandas(desc="Vectorizing text")
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = vectorizer.fit_transform(tqdm(data['posts']))

# Target labels
y = data['type']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Logistic Regression classifier
clf = LogisticRegression(max_iter=500, multi_class='ovr', n_jobs=-1)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluation
acc = accuracy_score(y_test, y_pred)
print(f"Classifier Accuracy: {acc:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=data['type'].unique())
cm_df = pd.DataFrame(cm, index=data['type'].unique(), columns=data['type'].unique())
print("\nConfusion Matrix:")
display(cm_df)


## 4. Exploratory Data Analysis (EDA)

### MBTI Dimension Distribution

*   See balance in I/E, S/N, T/F, J/P.






In [ ]:
dimension_stats = pd.DataFrame({
    "Dimension": ["I/E", "S/N", "T/F", "J/P"],
    "Majority": [
        "Introvert" if data["I_E"].mean() > 0.5 else "Extrovert",
        "Sensing" if data["S_N"].mean() > 0.5 else "Intuition",
        "Thinking" if data["T_F"].mean() > 0.5 else "Feeling",
        "Judging" if data["J_P"].mean() > 0.5 else "Perceiving"
    ],
    "Split": [
        f"{data['I_E'].mean():.1%} / {1-data['I_E'].mean():.1%}",
        f"{data['S_N'].mean():.1%} / {1-data['S_N'].mean():.1%}",
        f"{data['T_F'].mean():.1%} / {1-data['T_F'].mean():.1%}",
        f"{data['J_P'].mean():.1%} / {1-data['J_P'].mean():.1%}"
    ],
    "Balance_Score": [
        abs(data['I_E'].mean() - 0.5),
        abs(data['S_N'].mean() - 0.5),
        abs(data['T_F'].mean() - 0.5),
        abs(data['J_P'].mean() - 0.5)
    ]
})
print("\nClass Balance by Dimension (0.0 = perfect balance, 0.5 = max imbalance):")
display(dimension_stats)

The dataset shows varying levels of imbalance across MBTI dimensions. The I/E dimension leans heavily toward Introverts (76%), while the S/N dimension is extremely skewed toward Intuition (91%). T/F favors Thinking (65%), and J/P is relatively balanced. These imbalances may affect downstream classification performance and should be considered when evaluating model results.

### Class Imbalance Analysis

*   Quantify type imbalance





In [ ]:
type_counts = data['type'].value_counts()
print(f"\nMost common type: {type_counts.index[0]} ({type_counts.iloc[0]} posts)")
print(f"Least common type: {type_counts.index[-1]} ({type_counts.iloc[-1]} posts)")
print(f"Imbalance ratio: {type_counts.iloc[0] / type_counts.iloc[-1]:.1f}x")
print(f"Types with <100 samples: {(type_counts < 100).sum()} out of 16")
print(f"Types with <50 samples: {(type_counts < 50).sum()} out of 16")

The dataset is highly imbalanced at the individual MBTI type level. INTP is the most represented type with ~25k posts, whereas ESFJ has only 181 posts, yielding an imbalance ratio of ~138x. Fortunately, all types have at least 50 posts, so none are completely missing, but this skew should be kept in mind for model training and evaluation.

### N-gram Diversity Analysis

*   Summary stats for language patterns





In [ ]:
ngram_diversity = {}
for mbti_type in ngram_analysis.keys():
    bigrams = ngram_analysis[mbti_type]['bigrams']
    if len(bigrams) > 0:
        total_bigrams = sum([count for _, count in bigrams])
        ngram_diversity[mbti_type] = {
            'total_bigrams': total_bigrams,
            'unique_in_top10': len(bigrams),
            'top1_concentration': bigrams[0][1] / total_bigrams if total_bigrams > 0 else 0,
            'most_common': bigrams[0][0]
        }

ngram_df = pd.DataFrame(ngram_diversity).T
print("\nN-gram Diversity by Type:")
print("(Higher concentration = more repetitive language)")
display(ngram_df.sort_values('top1_concentration', ascending=False))

The N-gram analysis shows that while the top 10 bigrams are consistent across all types, the frequency of the most common bigram varies. ‘Feel like’ dominates for most types, reflecting a common informal phrasing in online posts. ESTP shows the lowest concentration of its top bigram, suggesting more lexical diversity, whereas INFP and INFJ have higher concentrations, indicating more repetitive language patterns.

### TF-IDF Feature Importance

*   Identify which words/bigrams are most predictive of each type.




In [ ]:
import numpy as np

print("\nTop 10 TF-IDF Features by MBTI Type:\n")
for idx, mbti_type in enumerate(clf.classes_):
    coefs = clf.coef_[idx]
    top_idx = np.argsort(coefs)[-10:]
    top_features = np.array(vectorizer.get_feature_names_out())[top_idx]
    top_weights = coefs[top_idx]

    print(f"{mbti_type}:")
    for feature, weight in zip(top_features, top_weights):
        print(f"  {feature}: {weight:.3f}")
    print()

TF-IDF analysis highlights the words most distinctive to each MBTI type. Self-references (e.g., ‘enfp’, ‘intj’) dominate, as expected in a dataset labeled by personality. Beyond labels, lexical patterns reflect type tendencies: ENFPs emphasize relational and positive words (‘friend’, ‘love’, ‘excite’), INTJs focus on cognitive or planning related terms (‘te’, ‘plan’, ‘efficient’), and ISTPs/ESTPs feature more concrete or action oriented words (‘se’, ‘military’, ‘fix’). This preliminary observation suggests linguistic markers could differentiate personality types.

## 5. Visualization and Insights

### MBTI Type Distribution

In [ ]:
plt.figure(figsize=(12,5))
sns.countplot(data=data, x='type', order=data['type'].value_counts().index, palette="viridis")
plt.xticks(rotation=45)
plt.title("Distribution of MBTI Types", fontsize=14, fontweight='bold')
plt.xlabel("MBTI Type")
plt.ylabel("Number of Posts")
plt.tight_layout()
plt.show()

## Dataset Overview: MBTI Posts Distribution

The dataset is heavily dominated by the four **Introverted-Intuitive (IN) types**:

| Type  | Approx. Posts |
|-------|---------------|
| INTP  | ~25,000       |
| INTJ  | ~22,500       |
| INFJ  | ~15,000       |
| INFP  | ~12,000       |

Other types, particularly **Extraverted-Sensing (ES)** types like ESFP, ESTP, and ESFJ, have very few posts (some < 500).  

### Observations
1. **Platform bias**: Data is sourced from websites like PersonalityCafe and Reddit, which attract users interested in **in depth discussions** and **anonymous interactions**. This skews the dataset toward **introverted-intuitive types**.
2. **Behavioral trends**: Extraverted-sensing types are more likely to use platforms like Instagram and Facebook, which emphasize **visual content, sensory experiences, and non-anonymous interactions**.
3. **Impact on modeling**: This imbalance can affect **model accuracy and fairness**, as predictions may overrepresent IN types while underrepresenting ES types.


### MBTI Dimension Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Map binary to descriptive labels
data_plot = data.copy()
data_plot["I_E_label"] = data_plot["I_E"].map({1:"Introvert", 0:"Extrovert"})
data_plot["S_N_label"] = data_plot["S_N"].map({1:"Sensing", 0:"Intuition"})
data_plot["T_F_label"] = data_plot["T_F"].map({1:"Thinking", 0:"Feeling"})
data_plot["J_P_label"] = data_plot["J_P"].map({1:"Judging", 0:"Perceiving"})

# Create a 2x2 grid of readable plots
fig, axes = plt.subplots(2, 2, figsize=(14,10))
sns.countplot(data=data_plot, x="I_E_label", ax=axes[0,0], palette="pastel")
axes[0,0].set_title("Introvert vs Extrovert")

sns.countplot(data=data_plot, x="S_N_label", ax=axes[0,1], palette="pastel")
axes[0,1].set_title("Sensing vs Intuition")

sns.countplot(data=data_plot, x="T_F_label", ax=axes[1,0], palette="pastel")
axes[1,0].set_title("Thinking vs Feeling")

sns.countplot(data=data_plot, x="J_P_label", ax=axes[1,1], palette="pastel")
axes[1,1].set_title("Judging vs Perceiving")

# Add percentage labels on top of bars
for ax in axes.flatten():
    total = len(data_plot)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2, height + 50, f"{height/total:.1%}",
                ha='center', fontsize=10)

plt.tight_layout()
plt.show()


### 1. Introvert vs Extrovert (I/E)
- **Introverts**: 76.1%  
- **Extroverts**: 23.9%  

The dataset is strongly **I-heavy**, reflecting its origins in long form online communities that naturally attract introverts. Linguistic patterns are therefore likely to be **introspective, analytical, and self focused**.

### 2. Sensing vs Intuition (S/N)
- **Intuition**: 91.3%  
- **Sensing**: 8.7%  

Extremely **N-dominant**, making S-type patterns underrepresented and less reliable. Most extracted linguistic features reflect **intuitive-style writing**, so ML models will likely struggle to detect S-types, resulting in the **lowest recall** along this dimension.

### 3. Thinking vs Feeling (T/F)
- **Thinking**: 65.2%  
- **Feeling**: 34.8%  

This is the **most balanced dimension**, with enough T and F posts to detect meaningful differences. Models should perform best here. We can analyze:
- **Analytical vs. emotional vocabulary**  
- **Argumentative vs. experiential phrasing**  
- **Logical vs. empathetic sentence structures**

### 4. Judging vs Perceiving (J/P)
- **Perceiving**: 58.1%  
- **Judging**: 41.9%  

Slightly skewed but not extreme. J/P linguistic differences remain **reliable for analysis**. P dominance aligns with the N/T dominance, reflecting that **NT and NP types dominate the dataset**.


### N-gram Frequency Visualization

In [ ]:
# Create subplots for all 16 types
fig, axes = plt.subplots(4, 4, figsize=(20, 16))
axes = axes.flatten()

for idx, mbti_type in enumerate(sorted(ngram_analysis.keys())):
    top_bigrams = ngram_analysis[mbti_type]['bigrams']
    if len(top_bigrams) > 0:
        words, counts = zip(*top_bigrams[:5])
        axes[idx].barh(list(words), list(counts), color=plt.cm.magma(idx/16))
        axes[idx].set_title(mbti_type, fontweight='bold', fontsize=12)
        axes[idx].invert_yaxis()
        axes[idx].set_xlabel("Frequency", fontsize=9)
    else:
        axes[idx].text(0.5, 0.5, 'No data', ha='center', va='center')
        axes[idx].set_title(mbti_type, fontweight='bold')

plt.suptitle("Top 5 Bigrams by MBTI Type", fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

Across nearly all 16 MBTI types, **“feel like”** dominates as the most frequent bigram, except for **ESTP**.  

- **“Feel like” is not an emotional expression** but a **discourse marker**.  
  It often signals:
  - **Uncertainty:** “I feel like this means…”
  - **Opinion introduction:** “I feel like people forget…”  
- Its reflects **casual, introspective online communication**, typical of the MBTI dataset community (Reddit and PersonalityCafe).  
- Because it appears so frequently across all types, **“feel like” is not a discriminative linguistic feature** for MBTI classification.

### Outlier: ESTP
- **ESTP** stands out with bigrams such as **“lol lol”**, indicating **more reactive and informal language**.  
- This aligns with typical personality descriptions of ESTPs as **action oriented** and **less introspective** compared to IN-dominant types.

### Additional Anomaly: ESTJ
- An unexpected pattern appears in **ESTJ** bigrams, where **“det er”** (Danish/Norwegian for *“it is”*) ranks among the most frequent.  
- This likely reflects **multilingual posts** or **sample-size sensitivity**, as ESTJ has relatively few entries.  
- Since bigram counts in smaller classes can be **skewed by a handful of outlier posts**, this highlights the need for **language filtering and careful dataset cleaning** before interpreting linguistic patterns.


Other common bigrams across types include **“sound like,” “look like,”** and **“make sense.”**  
These reflect a shared linguistic culture centered on **hedging** and **sensory comparison**, suggesting that stylistic norms, rather than personality, heavily shape expression within this dataset.

### Implications
- Simple **n-gram features** provide limited discriminative power for personality detection.  
- **Advanced linguistic features** are needed for more reliable classification.


### POS Ratio Heatmap

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(pos_summary, annot=True, cmap="YlGnBu", fmt=".2f", linewidths=0.5)
plt.title("Average POS Ratios by MBTI Type", fontsize=14, fontweight='bold')
plt.xlabel("POS Category", fontweight='bold')
plt.ylabel("MBTI Type", fontweight='bold')
plt.tight_layout()
plt.show()

POS ratios remain **remarkably consistent** across all MBTI types, showing minimal linguistic variation.

| POS Tag | Approx. Range |
|----------|----------------|
| NOUN | 0.46–0.55 |
| VERB | 0.21–0.25 |
| ADJ | 0.14–0.16 |
| ADV | 0.09–0.11 |
| PRON | ~0.02 |

- The **narrow range** across all types suggests that **POS distribution is largely constant**, indicating minimal stylistic difference at this grammatical level.
- **ESTJ** shows a slightly higher **noun ratio (~0.55)**, possibly reflecting a more **object-oriented or concrete linguistic focus**, which aligns with its personality profile.  
  However, this finding should be **interpreted cautiously** due to small sample size and the **earlier anomaly** observed in the ESTJ data.
- **Pronoun ratios** are very low (<0.02) across all types, not due to personality differences, but because **stopword removal** eliminated most pronoun tokens. As a result, pronouns **cannot be meaningfully compared** between types.

**POS ratios** do not vary significantly among personality types, they are **unlikely to be strong predictive features** for MBTI classification.  
Future models will require **semantic, lexical, or contextual features** to capture more meaningful distinctions in linguistic behavior.

### TF-IDF Feature Importance

In [ ]:
import numpy as np

fig, axes = plt.subplots(4, 4, figsize=(20, 18))
axes = axes.flatten()

for idx, mbti_type in enumerate(clf.classes_):
    coefs = clf.coef_[idx]
    top_idx = np.argsort(coefs)[-10:]
    top_features = np.array(vectorizer.get_feature_names_out())[top_idx]
    top_weights = coefs[top_idx]

    axes[idx].barh(top_features, top_weights, color=plt.cm.coolwarm(idx/16))
    axes[idx].set_title(mbti_type, fontweight='bold', fontsize=11)
    axes[idx].invert_yaxis()
    axes[idx].set_xlabel("Coefficient", fontsize=9)
    axes[idx].tick_params(axis='y', labelsize=8)

plt.suptitle("Top 10 TF-IDF Features by MBTI Type", fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

The visualization above preents the **top 10 TF-IDF features** for each MBTI type.  

Each bar indicates the words most characteristic of that type’s writing within the dataset.

- **ENFP**: Terms such as *“excite,” “friend,” “adventure,”* and *“dude”* reflect an **expressive and socially engaged language style**, consistent with their imaginative and outward  oriented traits.  
- **ENTP**: High occurrence of **casual and profane language** suggests a **spontaneous and informal communication pattern**, typical of discussion driven or debate heavy exchanges.  
- **INTP**: Frequent words like *“think,” “meme,”* and *“philosophy”* indicate **introspective and conceptual writing**, supporting their analytical and reflective tendencies.  
- **INTJ**: Keywords *“arrogant,” “plan,”* and *“efficient”* emphasize a **goal oriented, structured, and evaluative tone**, aligning with their reputation for systematic reasoning.  
- **INFJ**: The phrase *“door slam”* notably appears among top features, a phrase often used within MBTI communities to describe emotional boundary-setting, reflecting **self-protective or decisive language use**.  
- **INFP**: Words like *“dream,” “love,” “beautiful,”* and *“daydream”* demonstrate **emotionally charged and imaginative vocabulary**, reinforcing their association with creative expression.  
- **ISFJ**: Common terms such as *“friend,” “quiet,”* and *“boyfriend”* point to **relationship oriented and reserved language**, indicating a focus on personal rather than abstract themes.  
- **ESTP**: Bigrams like *“lol,” “haha,”* and *“yeah”* reflect **reactive and conversational writing**, matching their action-focused, socially dynamic tendencies.  
- **ESTJ**: Terms such as *“officer,” “germany,”* and *“police”* suggest **structured and factual content**, though interpretation should remain cautious due to **limited sample size** and **previous multilingual anomalies** (e.g., *“det er”*).  


TF-IDF captures **distinct lexical markers** for larger classes (especially INFP, ENFP, INTP), while smaller classes are more **susceptible to topic or language bias**.  


Personality differentiation appears more **semantic than structural** tone, vocabulary, and abstraction level distinguish types more than syntactic or POS features.


TF-IDF provides clear interpretability and highlights personality linked lexical tendencies.  
However, due to overlapping vocabulary and community specific slang, **semantic and contextual modeling** would be required for stronger classification performance.


## 6. Ethical Considerations

**Data Privacy:** All posts are publicly available and anonymized.

**Bias Awareness:** Overrepresentation of certain types (e.g., INFP, INTP) may affect balance.

**Language & Cultural Bias:** Dataset is English only and Reddit centric.

**Responsible Interpretation:** Avoid generalizing results to real world psychological profiling.

Ethical handling ensures that insights remain educational, non diagnostic, and respect user privacy.

## 7. Conclusion

The linguistic exploration of the MBTI text dataset reveals a strong demographic and skew toward introverted–intuitive types, particularly INTP, INTJ, INFJ, and INFP. This reflects the nature of the data sources platforms, which is as Reddit and PersonalityCafe. It favor users who engage in introspective and abstract self expression discussions. The dataset does not represent the full MBTI spectrum equally, which limits the generalizability of linguistic findings, especially for underrepresented sensing types.

Across the four MBTI dimensions, the S/N axis displays the most extreme imbalance (91% Intuitive), followed by I/E (76% Introverted). The T/F and J/P dimensions remain relatively balanced, and therefore more suitable for linguistic modeling. Any personality based classification task should account for these imbalances through resampling, class weighting, and targeted evaluation metrics.

At the linguistic level, n-gram and POS analyses indicate that surface level features, such as word frequency or POS distribution, vary minimally across types. Common phrases such as “feel like” dominate nearly all categories, suggesting a shared discourse style rooted in online communication norms rather than personality. Anomalies such as the ESTJ bigram “det er” underscore the importance of language filtering and sample size sensitivity in interpreting results. Similarly, POS ratios remain nearly constant (e.g. NOUN ≈ 0.46–0.55, VERB ≈ 0.21–0.25), confirming that grammatical structure is not a big differentiator among types.

In contrast, TF-IDF analysis highlights clear lexical distinctions at the semantic level. Terms associated with abstract reasoning (INTP, INTJ), emotional or imaginative expression (INFP, ENFP), and interpersonal focus (ISFJ, ESFJ) align with personality based expectations. These findings suggest that meaningful differentiation emerges from semantic content rather than syntactic form.

Overall, while the dataset supports broad associations between language and personality, its imbalance and community specific bias constrains the predictive reliability. Future work should instead prioritize balanced data collection, multilingual filtering, and the inclusion of context aware linguistic features to capture deeper psychological and semantic patterns beyond surface level text statistics.